In [1]:
# convert decimal lat / lon in WGS83 EPSG: 4326 to NAD83(NRS2007) UTM meters, Zone 17N EPSG:3724
# to get this to work, I had to define GDAL_DATA in my Win environment path
import numpy as np

# GDAL Python library
from osgeo import osr
# EXIF Reader
import exifread
# Image library
from PIL import Image

import matplotlib.pyplot as plt
get_ipython().magic(u'matplotlib inline')

src_crs = 'EPSG:4326'
dst_crs = 'EPSG:3724'

# source coordinate system
srs_cs = osr.SpatialReference()
srs_cs.SetFromUserInput(src_crs)

# destination coordinate system
dst_cs = osr.SpatialReference()
dst_cs.SetFromUserInput(dst_crs)

# osr image transformation object
transform = osr.CoordinateTransformation(srs_cs, dst_cs)

# print coordinate system information
print " >> SOURCE COORDINATE REFERENCE SYSTEM: "
print srs_cs

print " >> DESTINATION COORDINATE REFERENCE SYSTEM: "
print dst_cs


 >> SOURCE COORDINATE REFERENCE SYSTEM: 
GEOGCS["WGS 84",
    DATUM["WGS_1984",
        SPHEROID["WGS 84",6378137,298.257223563,
            AUTHORITY["EPSG","7030"]],
        AUTHORITY["EPSG","6326"]],
    PRIMEM["Greenwich",0,
        AUTHORITY["EPSG","8901"]],
    UNIT["degree",0.0174532925199433,
        AUTHORITY["EPSG","9122"]],
    AUTHORITY["EPSG","4326"]]
 >> DESTINATION COORDINATE REFERENCE SYSTEM: 
PROJCS["NAD83(NSRS2007) / UTM zone 17N",
    GEOGCS["NAD83(NSRS2007)",
        DATUM["NAD83_National_Spatial_Reference_System_2007",
            SPHEROID["GRS 1980",6378137,298.257222101,
                AUTHORITY["EPSG","7019"]],
            TOWGS84[0,0,0,0,0,0,0],
            AUTHORITY["EPSG","6759"]],
        PRIMEM["Greenwich",0,
            AUTHORITY["EPSG","8901"]],
        UNIT["degree",0.0174532925199433,
            AUTHORITY["EPSG","9122"]],
        AUTHORITY["EPSG","4759"]],
    PROJECTION["Transverse_Mercator"],
    PARAMETER["latitude_of_origin",0],
    PARAMETER["cen

In [17]:
fnin = "D:\\2017_Matthew\\maitane\\2016-10-30_control_points.csv"
f=open(fnin,'r')
header = f.readline()
print header
for line in f:
    line = line.strip()
    col = line.split(',')
    name = col[0]
    lat = float(col[1])+float(col[2])/60.+float(col[3])/3600.
    lon = -(float(col[4])+float(col[5])/60.+float(col[6])/3600.)
    print(name, lat, lon)
f.close

IOError: [Errno 2] No such file or directory: 'D:\\2017_Matthew\\maitane\\2016-10-30_control_points.csv'